In [1]:
import numpy as np
from hsmm_base import GaussianHSMM

In [2]:
# sample observations (from hsmmlearn)
obs = np.array([9.7471989, 9.01877392, -1.28428112, 0.55009463, 0.58425178, 0.43354177,
                10.63374352, 10.14025491, 5.3215164, 5.46421001, 5.16719792, 9.12216889,
                10.49171453, -0.72812025, 0.57309517, 0.3420868, -1.35338431, 4.12587557,
                6.907117, 5.41243634])

In [3]:
R = GaussianHSMM(n_states = 3, n_durations = 4)

In [4]:
# initial parameters (from hsmmlearn tutorial)

R.pi = np.array([1 / 3, 1 / 3, 1 / 3])

R.dur = np.array([
    [0.1, 0.005, 0.005, 0.89],
    [0.1, 0.005, 0.89, 0.005],
    [0.1, 0.89, 0.005, 0.005]
])

R.tmat = np.array([
        [0.0, 0.5, 0.5],
        [0.3, 0.0, 0.7],
        [0.6, 0.4, 0.0]
    ])

R.mean = np.array([0.0, 5.0, 10.0])
R.sdev = np.array([1, 1, 1])

In [5]:
# EM algorithm
R.fit(obs, censoring = 1)

FIT: reestimation complete for 1th loop.
FIT: reestimation complete for 2th loop.
FIT: reestimation complete for 3th loop.
FIT: reestimation complete for 4th loop.
FIT: reestimation complete for 5th loop.
FIT: converged at 6th loop.


In [6]:
# Log-probability of time-series under the model
R.score(obs, censoring = 1)

-25.88361518293564

In [7]:
# new parameters
print("Start Probabilities:\n", R.pi, "\n")
print("Transition Matrix:\n", R.tmat, "\n")
print("Durations:\n", R.dur, "\n")
print("Means:\n", R.mean, "\n")
print("Std. Deviations:\n", R.sdev)

Start Probabilities:
 [0.00000000e+000 3.32839523e-145 1.00000000e+000] 

Transition Matrix:
 [[0.         0.5        0.5       ]
 [0.00934733 0.         0.99065267]
 [0.66666667 0.33333333 0.        ]] 

Durations:
 [[4.13125038e-097 5.91296194e-134 4.79516454e-094 1.00000000e+000]
 [2.26320278e-059 2.54372682e-052 1.00000000e+000 5.18716842e-108]
 [4.84284483e-069 1.00000000e+000 1.53199290e-124 1.92271551e-148]] 

Means:
 [-0.11033944  5.39972554  9.85897578] 

Std. Deviations:
 [0.80554035 0.8123452  0.62464109]


In [8]:
# Viterbi algorithm
states, logprob = R.predict(obs, censoring = 1)

In [9]:
# correct states (from hsmmlearn tutorial)
correct = np.array([2,2,0,0,0,0,2,2,1,1,1,2,2,0,0,0,0,1,1,1])

In [10]:
# how many incorrect states?
np.sum(states != correct)

0